In [25]:
import os
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from facenet_pytorch import InceptionResnetV1

In [26]:
dataset_path = "../../../ChimpRec-Dataset/Chimpanzee_recognition_dataset"

transform = transforms.Compose([
    transforms.Resize((160, 160)),
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomRotation(10),  
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

train_dataset = datasets.ImageFolder(os.path.join(dataset_path, "train"), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(dataset_path, "val"), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=400, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=False, num_workers=4)

class_to_idx = train_dataset.class_to_idx
print("Classes :", class_to_idx)

Classes : {'FANLE': 0, 'FANWA': 1, 'FLANLE': 2, 'JEJE': 3, 'JOYA': 4, 'PELEY': 5}


In [27]:
facenet = InceptionResnetV1(pretrained='vggface2').eval()

On gèle toutes les couches sauf la dernière afin que les poids calculés pour les couches précédentes sur base de l'entraînement qui a été fait avec vggface restent les mêmes et qu'on ne modifie que la dernière couche:

In [28]:
# Geler toutes les couches sauf les 5 dernières
for param in facenet.parameters():
    param.requires_grad = False

"""for param in facenet.last_linear.parameters():
    param.requires_grad = True"""
for layer in list(facenet.children())[-3:]:
    for param in layer.parameters():
        param.requires_grad = True

In [29]:
"""num_classes = len(class_to_idx)
facenet.classify = True
facenet.last_linear = nn.Linear(facenet.last_linear.in_features, num_classes)"""

# Déplacer le modèle sur GPU si disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
facenet = facenet.to(device)

In [30]:
criterion = nn.CrossEntropyLoss()
#criterion = nn.TripletMarginLoss(margin=1.0)
optimizer = optim.Adam(facenet.parameters(), lr=0.001)

In [32]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        print("-----------------------")

        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            #Pour remettre les gradients à 0 car par défaut avec pytorch ils s'accumulent donc si on fait pas ça les 
            # gradients de cette étape d’entraînement s'ajouteront à ceux de l'étape précédente
            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            loss.backward() #calcule le gradients des param 
            optimizer.step() #optimise les param en fonction des gardients 

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects / len(train_loader.dataset)

        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # Phase de validation
        model.eval()
        val_loss = 0.0
        val_corrects = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

        val_loss /= len(val_loader.dataset)
        val_acc = val_corrects / len(val_loader.dataset)

        print(f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")


train_model(facenet, train_loader, val_loader, criterion, optimizer, num_epochs=20)

torch.save(facenet.state_dict(), "facenet_chimpanzee_finetuned.pth")
#print("Modèle sauvegardé.")



Epoch 1/20
-----------------------
Train Loss: 6.1975 Acc: 0.0574
Val Loss: 6.1612 Acc: 0.2833

Epoch 2/20
-----------------------
Train Loss: 6.1740 Acc: 0.1724
Val Loss: 6.1493 Acc: 0.3600

Epoch 3/20
-----------------------
Train Loss: 6.1473 Acc: 0.3217
Val Loss: 6.1264 Acc: 0.4367

Epoch 4/20
-----------------------
Train Loss: 6.1149 Acc: 0.4417
Val Loss: 6.0984 Acc: 0.4950

Epoch 5/20
-----------------------
Train Loss: 6.0849 Acc: 0.5233
Val Loss: 6.0752 Acc: 0.5483

Epoch 6/20
-----------------------
Train Loss: 6.0662 Acc: 0.5600
Val Loss: 6.0536 Acc: 0.5700

Epoch 7/20
-----------------------
Train Loss: 6.0578 Acc: 0.5717
Val Loss: 6.0295 Acc: 0.6100

Epoch 8/20
-----------------------
Train Loss: 6.0381 Acc: 0.5939
Val Loss: 6.0194 Acc: 0.6167

Epoch 9/20
-----------------------
Train Loss: 6.0277 Acc: 0.5974
Val Loss: 6.0114 Acc: 0.6467

Epoch 10/20
-----------------------
Train Loss: 6.0194 Acc: 0.6083
Val Loss: 6.0015 Acc: 0.6500

Epoch 11/20
-----------------------
Tr

Pour charger le modèle ensuite: 

In [35]:
facenet_loaded = InceptionResnetV1(pretrained='vggface2').eval()

facenet_loaded.load_state_dict(torch.load("facenet_chimpanzee_finetuned.pth", map_location=torch.device('cpu')))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
facenet_loaded = facenet_loaded.to(device)

In [41]:
from PIL import Image

# Charger et transformer l'image
def predict_image(image_path, model, transform, class_to_idx, device):
    model.eval()
    
    # Charger l'image
    image = Image.open(image_path).convert('RGB')
    
    # Appliquer les transformations
    image_tensor = transform(image).unsqueeze(0).to(device)  # Ajouter une dimension pour le batch
    
    # Effectuer une prédiction
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted_idx = torch.max(outputs, 1)  # Trouver l'indice de la classe prédite
    
    # Récupérer le label correspondant
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    predicted_class = idx_to_class[predicted_idx.item()]
    
    return predicted_class

# Exemple d'utilisation
image_path = "../../../ChimpRec-Dataset/Chimpanzee_recognition_dataset/val/JEJE/JEJE_12.jpg"
predicted_class = predict_image(image_path, facenet_loaded, transform, class_to_idx, device)
print(f"Classe prédite : {predicted_class}")


Classe prédite : FANWA
